In [2]:
import numpy as np
import pandas as pd
import keras
from pandas import read_csv
from pandas import datetime
from pandas import concat
from pandas import Series
from matplotlib import pyplot
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import __version__

Using TensorFlow backend.


In [3]:
init_notebook_mode(connected=True)

In [4]:
dataset = np.genfromtxt("Dataset/naia_international_01-16.csv", delimiter=",",skip_header=1,dtype="U75")
dataset = np.array(dataset) # The dataset from data.gov.ph containing the volume of passenger in the International airport of NAIA from 2014-2016.
pd.DataFrame(dataset)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2001,International,655103,535108,640082,704200,670212,606293,605089,605659,516740,483450,509153,612524,7143613
1,2002,International,626833,550378,681277,694193,687904,639544,621244,636119,547286,583248,555634,642719,7466379
2,2003,International,685582,566172,609512,533691,499787,512020,587782,622348,561158,615236,617045,716005,7126338
3,2004,International,761363,613616,704411,778538,758789,701166,720574,695636,605231,658856,661473,756351,8416004
4,2005,INT'L,783679,676199,802999,822869,851397,770319,777819,748956,665222,736257,745964,840326,9222006
5,2006,International,873170,744889,821993,891893,900743,798607,815099,793180,701551,760649,790407,874468,9766649
6,2007,International,922705,804397,937206,959647,966160,893639,911366,878313,789857,828626,875814,956985,10724715
7,2008,International,1006690,898974,1015186,1013034,1056383,945797,935579,900121,796794,884495,854208,966106,11273367
8,2009,International,997564,831362,978820,1052976,1020107,878236,907794,932771,799917,865481,920160,1018056,11203244
9,2010,International,1071113,915240,1097950,1121359,1151077,1043889,1039283,1016439,885721,966364,995624,1076542,12380601


In [5]:
# Transform and clean the data
delete_type = np.delete(dataset,1,1) # Remove type column
delete_total = np.delete(delete_type,13,1)
cleaned_dataset = pd.DataFrame(delete_total)

In [6]:
temp_row =[]
for row in delete_total:
    b= []
    for i in range(1,len(row)):
        a = [row[0],str(i),int(row[i])]
        b.append(a)  
    temp_row.append(b)     
temp_row      

final_row = []
for i in range(0,14):
    for row in temp_row[i]:
        final_row.append(row)
final_row
volume_row =[]
for i in final_row:
    volume_row.append(i[2])

In [8]:
final_df = pd.DataFrame(final_row)
final_df.columns = ['Year', 'Month','Volume']
final_df # final lists of dataset containing Year - Month - Volume of Passengers

,Year,Month,Volume
0,2001,1,655103
1,2001,2,535108
2,2001,3,640082
3,2001,4,704200
4,2001,5,670212
5,2001,6,606293
6,2001,7,605089
7,2001,8,605659
8,2001,9,516740
9,2001,10,483450


<p> The 168 item lists are the 14 year passenger volume data of the International airport of Ninoy Aquino International Airport. </p>

<p> Dataset structure: </p>

<center>[year, month, volume]</center>
<center>[2014, 1, 100000]</center>

| Header | Definition
|---------------------|--------------------
| `year` | Year  
| `month` | Month
| `volume` | Volume of passenger in the given date  

We will split the dataset into two parts: training set and test set.

## Training Set
</p> The first 13 years of the dataset (2001-11,2014-2015) will be taken for the training set </p>

In [9]:
training_set = final_row[0:156]
training_set
ts_df = pd.DataFrame(training_set)
ts_df.columns = ["Year","Month","Volume"]
ts_df

,Year,Month,Volume
0,2001,1,655103
1,2001,2,535108
2,2001,3,640082
3,2001,4,704200
4,2001,5,670212
5,2001,6,606293
6,2001,7,605089
7,2001,8,605659
8,2001,9,516740
9,2001,10,483450


## Test Set

<p> The remaining data which consists of data from 2016 will be taken for the test set. </p>

In [10]:
test_set= final_row[156:]
test_set
test_df = pd.DataFrame(test_set)
test_df.columns = ["Year","Month","Volume"]
test_df

,Year,Month,Volume
0,2016,1,1687391
1,2016,2,1483683
2,2016,3,1623942
3,2016,4,1677129
4,2016,5,1684676
5,2016,6,1544653
6,2016,7,1610728
7,2016,8,1515590
8,2016,9,1354769
9,2016,10,1460415


In [11]:
year = ['2001', '2002', '2003', '2004', '2005', '2006', '2007',
         '2008', '2009', '2010', '2011','2014','2015','2016']

year_01 = sum(volume_row[0:12])/12
year_02 = sum(volume_row[12:24])/12
year_03 = sum(volume_row[24:36])/12
year_04 = sum(volume_row[36:48])/12
year_05 = sum(volume_row[48:60])/12
year_06 = sum(volume_row[60:72])/12
year_07 = sum(volume_row[72:84])/12
year_08 = sum(volume_row[84:96])/12
year_09 = sum(volume_row[96:108])/12
year_10 = sum(volume_row[108:120])/12
year_11 = sum(volume_row[120:132])/12
year_14 = sum(volume_row[132:144])/12
year_15 = sum(volume_row[144:156])/12
year_16 = sum(volume_row[156:168])/12
year_val= [year_01,year_02,year_03,year_04,year_05,year_06,year_07,year_08,year_09,year_10,year_11,year_14,
          year_15,year_16]

trace0 = go.Scatter(
    x = year,
    y = year_val,
    name = 'Passenger Volume',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)
data = [trace0]
# Edit the layout
layout = dict(title = 'Average NAIA Number of Passengers in NAIA 2001-2011, 2014-2016',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Passenger Volume'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-line')

<h3>LSTM Recurrent Neural Network Data Preparation </h3>
<p> This section is divided into three steps: </p>


<ol>1) Transform the time series into a supervised learning problem </ol> 

<ol> 2) Transform the time series data so that it is stationary </ol>
<ol> 3) Transform the observations to have a specific scale </ol>


<h6> Transform the time series into a supervised learning problem </h6>

In [12]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [13]:
supervised = timeseries_to_supervised(volume_row,1)

<h6> Transform time series to stationary </h6>

In [14]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted values
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [15]:
# transform to be stationary
differenced = difference(volume_row,1)
# invert transform
inverted = list()
for i in range(len(differenced)):
    value = inverse_difference(volume_row, differenced[i],len(volume_row)-i)
    inverted.append(value)
inverted = Series(inverted)

<h6>Transform time series to scale </h6>

In [16]:
# transform scale
X = Series(volume_row)
X = X.reshape(len(X), 1)
scaler = MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)

# invert transform
inverted_X = scaler.inverse_transform(scaled_X)
inverted_series = Series(inverted_X[:,0])

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype int64 was converted to float64 by MinMaxScaler.



<h4> LSTM Recurrent Neural Network Model Development </h4>

In [17]:
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='mean_squared_error', optimizer="adam")
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

<h4>LSTM Recurrent Neural Network Forecast </h4>

In [18]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

## LSTM Recurrent Neural Network Simulation 

In [19]:
# get values from dataset
raw_values = volume_row
#print(len(raw_values))
# transform data to be stationary
diff_values = difference(raw_values, 1)
#print(diff_values)
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values
#print(len(supervised_values))
# split data into train and test sets
train,test = supervised_values[0:-12], supervised_values[-12:]

In [20]:
scaler, train_scaled, test_scaled = scale(train, test)

In [21]:
# fit the model
lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
    # make one-stop forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)
    # invert scaling 
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    predictions.append(yhat)
    expected = raw_values[len(train)+ i + 1]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1,yhat,expected))
    
# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:],predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
#pyplot.plot(raw_values[-12:], label="Expected Values")
#pyplot.plot(predictions, label="Predicted Values")
#pyplot.legend(loc='upper center', shadow=True)
#pyplot.show()

month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']

trace0 = go.Scatter(
    x = month,
    y = raw_values[-12:],
    name = 'Expected',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

trace1 = go.Scatter(
    x = month,
    y = predictions,
    name = 'Predicted',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)

data = [trace0,trace1]
# Edit the layout
layout = dict(title = 'Predicted vs Expected Number of Passengers in 2016',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Passenger Volume'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-line')


Month=1, Predicted=1646380.500011, Expected=1687391.000000
Month=2, Predicted=1701471.500011, Expected=1483683.000000
Month=3, Predicted=1497763.500011, Expected=1623942.000000
Month=4, Predicted=1638022.500011, Expected=1677129.000000
Month=5, Predicted=1691209.500011, Expected=1684676.000000
Month=6, Predicted=1698756.500011, Expected=1544653.000000
Month=7, Predicted=1558733.500011, Expected=1610728.000000
Month=8, Predicted=1624808.500011, Expected=1515590.000000
Month=9, Predicted=1529670.500011, Expected=1354769.000000
Month=10, Predicted=1368849.500011, Expected=1460415.000000
Month=11, Predicted=1474495.500011, Expected=1521530.000000
Month=12, Predicted=1535610.500011, Expected=1773862.000000
Test RMSE: 130044.065
